# Warsztaty Python w Data Science

***

## Data Wrangling z Biblioteką Pandas 
### Część 5 z 5 - Wszystkie sztuczki razem

---

# https://grouplens.org/datasets/movielens/

## Small: 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Last updated 9/2018.


## Pandas Chaining

***

# https://github.com/Bits-of-Data-PL/PythonDataScience

---

In [ ]:
import pandas as pd

movies = pd.read_csv('data/ml-latest-small/movies.csv')
movies

In [ ]:
movies[movies['genres']=='(no genres listed)']

In [ ]:
movies= movies[movies['genres']!='(no genres listed)']


In [ ]:
movies[movies['genres'].str.contains('-')]

In [ ]:
movies.genres = movies.genres.apply(lambda genre: genre.replace('-',''))
movies[movies['genres'].str.contains('-')]

In [ ]:
movies.genres.str.split('|')

In [ ]:
genres = movies.genres.str.split('|', expand=True)
genres

In [ ]:
movies_with_genres = pd.concat([movies, genres], axis=1)
movies_with_genres

In [ ]:
movies_with_genres.drop('genres', axis=1, inplace=True)
movies_with_genres

In [ ]:
movies_with_genres.melt(id_vars=['movieId', 'title'], value_name='genre')

In [ ]:
genres = movies_with_genres.melt(id_vars=['movieId', 'title'], value_name='feature').drop('variable', axis=1)
genres

In [ ]:
genres['feature'].unique()

In [ ]:
genres

In [ ]:
genres[genres.movieId==1]

In [ ]:
movies_with_genres.melt(id_vars=['movieId', 'title'], value_name='feature').drop('variable', axis=1).dropna().reset_index(drop=True)

In [ ]:
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
ratings.head(10)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.style.use("dark_background")

plt.hist(ratings['rating'], 10,
         density=True,
         histtype='bar',
         facecolor='b',
         alpha=0.9)

plt.show()

In [ ]:
ratings = ratings.drop('timestamp', axis=1)

In [ ]:
ratings

In [ ]:
melted_movies_with_genres = movies_with_genres.melt(id_vars=['movieId', 'title'], value_name='feature').drop(['variable', 'title'], axis=1).dropna().reset_index(drop=True)
melted_movies_with_genres.columns

In [ ]:
melted_movies_with_genres = movies_with_genres.melt(id_vars=['movieId', 'title'], value_name='feature').drop(['variable', 'title'], axis=1).dropna().reset_index(drop=True)
melted_movies_with_genres.feature = melted_movies_with_genres.feature.apply(lambda feat: feat.replace('-',''))
ratings_and_movies = pd.merge(ratings, melted_movies_with_genres, on='movieId', how='inner')
ratings_and_movies

In [ ]:
grouped_features = ratings_and_movies.drop('movieId', axis=1).groupby(['userId', 'feature']).sum()
grouped_features

In [ ]:
grouped_features.reset_index(inplace=True)

In [ ]:
df.columns

In [ ]:
grouped_features.pivot(index='userId', columns=['feature'])['rating']

In [ ]:
grouped_features.pivot(index='userId', columns=['feature'])['rating'].fillna(0).reset_index()

In [ ]:
df = grouped_features.reset_index().pivot(index='userId', columns=['feature'])['rating'].fillna(0).reset_index()
df

In [ ]:
df["total"] = df.drop("userId", axis=1).sum(axis=1)

In [ ]:
df

In [ ]:
df.iloc[:,1:] = df.iloc[:,1:].div(df.total, axis=0)

In [ ]:
df

In [ ]:
user2 = df[df.userId==2].melt().drop(df. index[0])
user2.style.format({'value':'{:,.2%}'})

---

# Pandas Chaining

In [ ]:
grouped_features.reset_index().pivot(index='userId', columns=['feature'])['rating'].fillna(0).reset_index().assign(total=lambda d: d.drop("userId", axis=1).sum(axis=1))

In [ ]:
grouped_features.reset_index(
               ).pivot(index='userId', columns=['feature'])['rating']\
                .fillna(0
               ).reset_index(
               ).assign(total=lambda d: d.drop("userId", axis=1).sum(axis=1)
               ).set_index('userId'
               ).pipe(lambda df: df.iloc[:,1:].div(df.total, axis=0)
               ).reset_index()

In [ ]:
(grouped_features
               .reset_index()
               .pivot(index='userId', columns=['feature'])['rating']
               .fillna(0)
               .reset_index()
               .assign(total=lambda d: d.drop("userId", axis=1)
                                        .sum(axis=1))
               .set_index('userId')
               .pipe(lambda df: df.iloc[:,1:].div(df.total, axis=0))
               .reset_index())

In [ ]:
(ratings_and_movies.drop('movieId', axis=1)
                        .groupby(['userId', 'feature'])
                        .sum()
                        .reset_index()
                        .pivot(index='userId', columns=['feature'])['rating']
                        .fillna(0)
                        .reset_index()
                        .assign(total=lambda d: d.drop("userId", axis=1)
                                                 .sum(axis=1))
                        .set_index('userId')
                        .pipe(lambda df: df.iloc[:,1:]
                                           .div(df.total, axis=0))
                        .reset_index())

In [ ]:
(pd.merge(
    pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1), 
    melted_movies_with_genres, on='movieId', how='inner'
  ).drop('movieId', axis=1)
   .groupby(['userId', 'feature'])
   .sum()
   .reset_index()
   .pivot(index='userId', columns=['feature'])['rating']
   .fillna(0)
   .reset_index()
   .assign(total=lambda d: d.drop("userId", axis=1)
                            .sum(axis=1)
   ).set_index('userId')
    .pipe(lambda df: df.iloc[:,1:]
                       .div(df.total, axis=0))
    .reset_index())

In [ ]:
(pd.merge(
    pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1), 
    pd.concat([movies, movies.genres.str.split('|', expand=True)], axis=1)
             .drop('genres', axis=1)
             .melt(id_vars=['movieId', 'title'], value_name='feature')
             .assign(feature=lambda df: df.feature.replace('-',''))
             .drop(['variable', 'title'], axis=1)
             .dropna()
             .reset_index(drop=True), 
    on='movieId', how='inner'
)
  .drop('movieId', axis=1)
  .groupby(['userId', 'feature'])
  .sum()
  .reset_index()
  .pivot(index='userId', columns=['feature'])['rating']
  .fillna(0)
  .reset_index()
  .assign(total=lambda d: d.drop("userId", axis=1)
                           .sum(axis=1))
  .set_index('userId')
  .pipe(lambda df: df.iloc[:,1:]
                     .div(df.total, axis=0))
  .reset_index()
)

In [ ]:
import pandas as pd

pd.read_csv('data/ml-latest-small/movies.csv').pipe(lambda df:  pd.concat([df, df.genres.str.split('|', expand=True)], axis=1))

In [ ]:
(pd.merge(
    pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1), 
    pd.read_csv('data/ml-latest-small/movies.csv').pipe(lambda df:
                                                        df[df['genres']!='(no genres listed)'])
                                                  .assign(genres=lambda df: df.genres.apply(
                                                                                    lambda genre: genre.replace('-','')
                                                                             ) 
                                                         )
                                                  .pipe(lambda df:  
                                                        pd.concat([df, 
                                                                   df.genres.str.split('|', expand=True)], axis=1))
             .drop('genres', axis=1)
             .melt(id_vars=['movieId', 'title'], value_name='feature')
             .assign(feature=lambda df: df.feature.replace('-',''))
             .drop(['variable', 'title'], axis=1)
             .dropna()
             .reset_index(drop=True), 
    on='movieId', how='inner'
)
  .drop('movieId', axis=1)
  .groupby(['userId', 'feature'])
  .sum()
  .reset_index()
  .pivot(index='userId', columns=['feature'])['rating']
  .fillna(0)
  .reset_index()
  .assign(total=lambda d: d.drop("userId", axis=1)
                           .sum(axis=1))
  .set_index('userId')
  .pipe(lambda df: df.iloc[:,1:]
                     .div(df.total, axis=0))
  .reset_index()
)

In [ ]:
(pd.merge(
    pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1), 
    pd.read_csv('data/ml-latest-small/movies.csv').pipe(lambda df:
                                                        df[df['genres']!='(no genres listed)'])
                                                  .assign(genres=lambda df: df.genres.apply(
                                                                                    lambda genre: genre.replace('-','')
                                                                             ) 
                                                         )
                                                  .pipe(lambda df:  
                                                        pd.concat([df, 
                                                                   df.genres.str.split('|', expand=True)], axis=1))
             .drop('genres', axis=1)
             .melt(id_vars=['movieId', 'title'], value_name='feature')
             .assign(feature=lambda df: df.feature.replace('-',''))
             .drop(['variable', 'title'], axis=1)
             .dropna()
             .reset_index(drop=True), 
    on='movieId', how='inner'
)
  .drop('movieId', axis=1)
  .groupby(['userId', 'feature'])
  .sum()
  .reset_index()
  .pivot(index='userId', columns=['feature'])['rating']
  .fillna(0)
  .reset_index()
  .assign(total=lambda d: d.drop("userId", axis=1)
                           .sum(axis=1))
  .set_index('userId')
  .pipe(lambda df: df.div(df.total, axis=0))
  .assign(**{k : lambda df, col=k: df[col].mul(100.0) for k in ['Action', 'Adventure', 'SciFi', 'Thriller', 'Animation', 'Children', 'Comedy', 'Fantasy', 'Documentary'
                                                          'Drama', 'Crime', 'Horror', 'Western', 'Musical', 'Mystery', 'IMAX', 'Romance', 'War', 'FilmNoir', 'total',]   })
  .pipe(lambda df: df.round(2))
  .reset_index()
)

In [ ]:
pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1)

In [ ]:
df = pd.read_csv('data/ml-latest-small/ratings.csv')
df.drop("timestamp", axis=1, inplace=True)
df

In [ ]:
movies = pd.read_csv('data/ml-latest-small/movies.csv')
movies = movies[movies['genres']!='(no genres listed)']
genres = movies.genres.str.split('|', expand=True)
movies_with_genres = pd.concat([movies, genres], axis=1).drop('genres', axis=1)
ratings = pd.read_csv('data/ml-latest-small/ratings.csv')
ratings = ratings.drop('timestamp', axis=1)

melted_movies_with_genres = movies_with_genres.melt(id_vars=['movieId', 'title'], value_name='feature').drop(['variable', 'title'], axis=1).dropna().reset_index(drop=True)
melted_movies_with_genres.feature = melted_movies_with_genres.feature.apply(lambda feat: feat.replace('-',''))

ratings_and_movies = pd.merge(ratings, melted_movies_with_genres, on='movieId', how='inner')
ratings_and_movies

grouped_features = ratings_and_movies.drop('movieId', axis=1).groupby(['userId', 'feature']).agg(['sum', 'count'])
grouped_features

In [ ]:
(pd.merge(
    pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1), 
    pd.read_csv('data/ml-latest-small/movies.csv').pipe(lambda df:
                                                        df[df['genres']!='(no genres listed)'])
                                                  .assign(genres=lambda df: df.genres.apply(
                                                                                    lambda genre: genre.replace('-','')
                                                                             ) 
                                                         )
                                                  .pipe(lambda df:  
                                                        pd.concat([df, 
                                                                   df.genres.str.split('|', expand=True)], axis=1))
             .drop('genres', axis=1)
             .melt(id_vars=['movieId', 'title'], value_name='feature')
             .assign(feature=lambda df: df.feature.replace('-',''))
             .drop(['variable', 'title'], axis=1)
             .dropna()
             .reset_index(drop=True), 
    on='movieId', how='inner'
)
  .drop('movieId', axis=1)
  .groupby(['userId', 'feature'])
  .agg(['sum', 'count'])
  .reset_index()
  .pivot(index='userId', columns=['feature'])['rating']
  .fillna(0)
  .reset_index()
)

In [ ]:
(pd.merge(
    pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1), 
    pd.read_csv('data/ml-latest-small/movies.csv').pipe(lambda df:
                                                        df[df['genres']!='(no genres listed)'])
                                                  .assign(genres=lambda df: df.genres.apply(
                                                                                    lambda genre: genre.replace('-','')
                                                                             ) 
                                                    )
                                                  .pipe(lambda df:  
                                                        pd.concat([df, 
                                                                   df.genres.str.split('|', expand=True)], axis=1))
             .drop('genres', axis=1)
             .melt(id_vars=['movieId', 'title'], value_name='feature')
             .assign(feature=lambda df: df.feature.replace('-',''))
             .drop(['variable', 'title'], axis=1)
             .dropna()
             .reset_index(drop=True), 
    on='movieId', how='inner'
)
  .drop('movieId', axis=1)
  .groupby(['userId', 'feature'])
  .agg(['sum', 'count'])
  .reset_index()
  .pivot(index='userId', columns=['feature'])['rating']
  .fillna(0)
  .reset_index()
  .pipe( lambda df: df.astype({k:'int' for k in df.columns[1+len(df.columns)//2:] }))
  .pipe( lambda df: df.set_axis(df.columns.to_flat_index(), axis=1))
  .rename( mapper = lambda ind: "_".join(ind[::-1]) , axis=1)
  .assign(**{"total_sum": lambda d: d.iloc[:, 1:1+len(d.columns)//2].sum(axis=1),
             "total_count": lambda d: d.iloc[:, 2+len(d.columns)//2:-1].sum(axis=1)
            })
)

In [ ]:
(pd.merge(
    pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1), 
    pd.read_csv('data/ml-latest-small/movies.csv').pipe(lambda df:
                                                        df[df['genres']!='(no genres listed)'])
                                                  .assign(genres=lambda df: df.genres.apply(
                                                                                    lambda genre: genre.replace('-','')
                                                                             ) 
                                                    )
                                                  .pipe(lambda df:  
                                                        pd.concat([df, 
                                                                   df.genres.str.split('|', expand=True)], axis=1))
             .drop('genres', axis=1)
             .melt(id_vars=['movieId', 'title'], value_name='feature')
             .assign(feature=lambda df: df.feature.replace('-',''))
             .drop(['variable', 'title'], axis=1)
             .dropna()
             .reset_index(drop=True), 
    on='movieId', how='inner'
)
  .drop('movieId', axis=1)
  .groupby(['userId', 'feature'])
  .agg(['sum', 'count'])
  .reset_index()
  .pivot(index='userId', columns=['feature'])['rating']
  .fillna(0)
  .reset_index()
  .pipe( lambda df: df.astype({k:'int' for k in df.columns[1+len(df.columns)//2:] }))
  .pipe( lambda df: df.set_axis(df.columns.to_flat_index(), axis=1))
  .rename( mapper = lambda ind: "_".join(ind[::-1]) , axis=1)
  .assign(**{"total_sum": lambda d: d.filter(like='_sum', axis=1).sum(axis=1),
             "total_count":  lambda d: d.filter(like='_count', axis=1).sum(axis=1)
            })
)

In [ ]:
(pd.merge(
    pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1), 
    pd.read_csv('data/ml-latest-small/movies.csv').pipe(lambda df:
                                                        df[df['genres']!='(no genres listed)'])
                                                  .assign(genres=lambda df: df.genres.apply(
                                                                                    lambda genre: genre.replace('-','')
                                                                             ) 
                                                    )
                                                  .pipe(lambda df:  
                                                        pd.concat([df, 
                                                                   df.genres.str.split('|', expand=True)], axis=1))
             .drop('genres', axis=1)
             .melt(id_vars=['movieId', 'title'], value_name='feature')
             .assign(feature=lambda df: df.feature.replace('-',''))
             .drop(['variable', 'title'], axis=1)
             .dropna()
             .reset_index(drop=True), 
    on='movieId', how='inner'
)
  .drop('movieId', axis=1)
  .groupby(['userId', 'feature'])
  .agg(['sum', 'count'])
  .reset_index()
  .pivot(index='userId', columns=['feature'])['rating']
  .fillna(0)
  .reset_index()
  .pipe( lambda df: df.astype({k:'int' for k in df.columns[1+len(df.columns)//2:] }))
  .pipe( lambda df: df.set_axis(df.columns.to_flat_index(), axis=1))
  .rename( mapper = lambda ind: "_".join(ind[::-1]) , axis=1)
  .assign(**{"total_sum": lambda d: d.filter(like='_sum', axis=1).sum(axis=1),
             "total_count":  lambda d: d.filter(like='_count', axis=1).sum(axis=1)
            })
).columns

In [ ]:
result = (pd.merge(
    pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1), 
    pd.read_csv('data/ml-latest-small/movies.csv').pipe(lambda df:
                                                        df[df['genres']!='(no genres listed)'])
                                                  .assign(genres=lambda df: df.genres.apply(
                                                                                    lambda genre: genre.replace('-','')
                                                                             ) 
                                                    )
                                                  .pipe(lambda df:  
                                                        pd.concat([df, 
                                                                   df.genres.str.split('|', expand=True)], axis=1))
             .drop('genres', axis=1)
             .melt(id_vars=['movieId', 'title'], value_name='feature')
             .assign(feature=lambda df: df.feature.replace('-',''))
             .drop(['variable', 'title'], axis=1)
             .dropna()
             .reset_index(drop=True), 
    on='movieId', how='inner'
)
  .drop('movieId', axis=1)
  .groupby(['userId', 'feature'])
  .agg(['sum', 'count'])
  .reset_index()
  .pivot(index='userId', columns=['feature'])['rating']
  .fillna(0)
  .reset_index()
  .pipe( lambda df: df.astype({k:'int' for k in df.columns[1+len(df.columns)//2:] }))
  .pipe( lambda df: df.set_axis(df.columns.to_flat_index(), axis=1))
  .rename( mapper = lambda ind: "_".join(ind[::-1]) , axis=1)
  .assign(**{"total_sum": lambda d: d.filter(like='_sum', axis=1).sum(axis=1),
             "total_count":  lambda d: d.filter(like='_count', axis=1).sum(axis=1)})
  .assign(**{k : lambda df, col=k: df[col].div(df.total_sum) for k in [l+'_sum' for l in ['Action', 'Adventure', 'SciFi', 'Thriller', 'Animation', 'Children', 'Comedy', 'Fantasy', 
                                                          'Drama', 'Crime', 'Horror', 'Western', 'Musical', 'Mystery', 'IMAX', 'Romance', 'War', 'FilmNoir', 'total']]})
)
result

In [ ]:
(pd.concat([result.iloc[1:2,:].melt().iloc[1:-21,:].reset_index(drop=True), 
                result.iloc[1:2,:].melt().iloc[20:,:].reset_index(drop=True).rename(columns={x:x+"_c" for x in result.iloc[1:2,:].melt().columns})], axis=1)
    
  .pipe(lambda d: d.append(d.sum(numeric_only=True), ignore_index=True))

)

In [ ]:
result = (pd.merge(
    pd.read_csv('data/ml-latest-small/ratings.csv').drop("timestamp", axis=1), 
    pd.read_csv('data/ml-latest-small/movies.csv').pipe(lambda df:
                                                        df[df['genres']!='(no genres listed)'])
                                                  .assign(genres=lambda df: df.genres.apply(
                                                                                    lambda genre: genre.replace('-','')
                                                                             ) 
                                                    )
                                                  .pipe(lambda df:  
                                                        pd.concat([df, 
                                                                   df.genres.str.split('|', expand=True)], axis=1))
             .drop('genres', axis=1)
             .melt(id_vars=['movieId', 'title'], value_name='feature')
             .assign(feature=lambda df: df.feature.replace('-',''))
             .drop(['variable', 'title'], axis=1)
             .dropna()
             .reset_index(drop=True), 
    on='movieId', how='inner'
)
  .drop('movieId', axis=1)
  .groupby(['userId', 'feature'])
  .agg(['sum', 'count'])
  .reset_index()
  .pivot(index='userId', columns=['feature'])['rating']
  .fillna(0)
  .reset_index()
  .pipe( lambda df: df.astype({k:'int' for k in df.columns[1+len(df.columns)//2:] }))
  .pipe( lambda df: df.set_axis(df.columns.to_flat_index(), axis=1))
  .rename( mapper = lambda ind: "_".join(ind[::-1]) , axis=1)
  .assign(**{"total_sum": lambda d: d.filter(like='_sum', axis=1).sum(axis=1),
             "total_count":  lambda d: d.filter(like='_count', axis=1).sum(axis=1)})
  .assign(**{k : lambda df, col=k: 100.0*df[col].div(df.total_sum) for k in [l+'_sum' for l in ['Action', 'Adventure', 'SciFi', 'Thriller', 'Documentary',  'Animation', 'Children', 'Comedy', 'Fantasy', 
                                                          'Drama', 'Crime', 'Horror', 'Western', 'Musical', 'Mystery', 'IMAX', 'Romance', 'War', 'FilmNoir', 'total',]]})
  .assign(**{k : lambda df, col=k: 100.0*df[col].div(df.total_count) for k in [l+'_count' for l in ['Action', 'Adventure', 'SciFi', 'Thriller', 'Documentary', 'Animation', 'Children', 'Comedy', 'Fantasy', 
                                                          'Drama', 'Crime', 'Horror', 'Western', 'Musical', 'Mystery', 'IMAX', 'Romance', 'War', 'FilmNoir', 'total']]})
  .pipe(lambda df: df.round(2))
)
result

In [ ]:
(pd.concat([result.iloc[1:2,:].melt().iloc[1:-21,:].reset_index(drop=True), 
                result.iloc[1:2,:].melt().iloc[20:-2,:].reset_index(drop=True).rename(columns={x:x+"_c" for x in result.iloc[1:2,:].melt().columns})], axis=1)
    
  .pipe(lambda d:d.append(d.sum(numeric_only=True), ignore_index=True))

)